In [2]:
from Bio import Entrez
from Bio import SeqIO

# Set your email (required by NCBI)
Entrez.email = "smrithisurender@berkely.edu"

# NCBI link or Accession ID
ncbi_link = "https://www.ncbi.nlm.nih.gov/nuccore/PP945018.1?report=genbank"  # Replace with your link
accession_id = ncbi_link.split('/')[-1].split('?')[0]  # Extract clean accession ID (e.g., PP945018.1)

# Fetch the sequence
handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="fasta", retmode="text")
record = SeqIO.read(handle, "fasta")
handle.close()

# Display the sequence in the notebook
print(f"Sequence ID: {record.id}")
print(f"Description: {record.description}")
print(f"Sequence:\n{record.seq}")


Sequence ID: PP945018.1
Description: PP945018.1 Flaviviridae sp. strain China-NX085, complete genome
Sequence:
TGTTGAGCGTTCCTTATCGGTGGTGTTTTCTCTGCGTTCCCTACTTTTTCGTTTGTTCACGTGTTCCGGCCGTCTTCACTTTAGACGCTAGCTCATTTCTTAATCGAGCTCGCGGCCTTTACTTACTACTGCTCCCGCCTTTAGGCGGGGGCCTCACACTCCAGACTTCTTGTGGTATTTGTTCTAACCCACTTTCCAACAAGATGTATACTCTTATCACAATCTCATCACTTTACTTCCACTTCTCTACTGCCCTTTTGGGCAACAACTCCGCAGCGTTTTGGTTTCCAGACGCTGACGGCCTCGACATGCCCCGCTGGCTAGCCCAGCAAGGGTACAACTACTCGTTTCACGGAGTGTGGAACATCCCCTCCTCCCATCACCCAGACTGTGAGCCCGAAGGGTTCGACAGTTGGGTCAAGGACAGGTGGCGTGGGCTTCTGTGGTTTGACAGCTCCCCAGAGACCAACCACACGTGTTGCCGCCTCCAGCACCACGAATACCTGAAGCATGCCTGTCGCGCTCAGGTGTCCCTGGAAGACTACCTGGCCGTCATCACGCACGCCCCCACCCCGGTCACGTCCCCCTTCTGCGCACGCCAGTGCCGCTTTCACGAAGCGGTCAACGTTGTGCAGGAAGCCGTTGATGCTGGCAGCTGGCACCCGTCGGACATCGTTGGGTGTAAAAAAGACGCGTGGTATCGCTTCGGCGACCACCTCGTTCCCCAGCCCTGCCTTCCCGACCCCCGCCCGACCATCGGCCCCACCCACCTCCCGCTGGGCCCCACTGACACGGGCCCCGGCATGGCCAAGGTCGCCCTCGAAAGCGTGACCGGCGTTGCCGCAGTCGCCGCGGTTGTGGCTCTGCTCTTCGTCCTCCGAACCAAGAG

In [8]:
from bs4 import BeautifulSoup
import requests
from Bio import Entrez, SeqIO
import pandas as pd

# Set your email (required by NCBI)
Entrez.email = "smrithisurender@berkely.edu"

# Main NCBI page URL (replace with your page URL)
main_page_url = "https://www.ncbi.nlm.nih.gov/nuccore?term=%28%28%22Flaviviridae%22%5BOrganism%5D%20OR%20%28%22Flaviviridae%22%5BOrganism%5D%20OR%20Flaviviridae%5BAll%20Fields%5D%29%29%20AND%20%28complete%5BAll%20Fields%5D%20AND%20genome%5BAll%20Fields%5D%29%29%20AND%20%28%22Homo%20sapiens%22%5BOrganism%5D%20OR%20human%5BAll%20Fields%5D%29%20&cmd=DetailsSearch"

# Function to extract genome links from the main page
def extract_genome_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all links on the page that lead to genome records
    links = [
        "https://www.ncbi.nlm.nih.gov" + a["href"]
        for a in soup.find_all("a", href=True)
        if "/nuccore/" in a["href"]
    ]
    return links

# Fetch sequences for all extracted links and create a table
def fetch_sequences_table(links):
    data = []  # List to store rows of the table
    for link in links:
        try:
            # Extract accession ID from the link
            accession_id = link.split('/')[-1].split('?')[0]
            
            # Fetch the sequence
            handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="fasta", retmode="text")
            record = SeqIO.read(handle, "fasta")
            handle.close()
            
            # Add a row to the table
            data.append({
                "Sequence ID": record.id,
                "Description": record.description,
                "Sequence": str(record.seq)
            })
        except Exception as e:
            print(f"Error fetching from {link}: {e}")
    # Convert the list of rows to a DataFrame
    return pd.DataFrame(data)

# Extract links from the main page
genome_links = extract_genome_links(main_page_url)

# Fetch sequences and create a table
sequences_table = fetch_sequences_table(genome_links)

# Print the table size
print(f"Table Size: {sequences_table.shape}")  # Number of rows and columns

sequences_table.to_csv("genome_sequences.csv", index=False)

Error fetching from https://www.ncbi.nlm.nih.gov/nuccore/advanced: HTTP Error 400: Bad Request
Error fetching from https://www.ncbi.nlm.nih.gov/sites/myncbi/nuccore/filters: HTTP Error 400: Bad Request
Error fetching from https://www.ncbi.nlm.nih.gov/sites/myncbi/nuccore/filters: HTTP Error 400: Bad Request
Error fetching from https://www.ncbi.nlm.nih.gov/nuccore/details?querykey=1: HTTP Error 400: Bad Request
Table Size: (80, 3)


In [9]:
from bs4 import BeautifulSoup
import requests
from Bio import Entrez, SeqIO
import pandas as pd
import time

# Set your email (required by NCBI)
Entrez.email = "smrithisurender@berkely.edu"

# Refined NCBI search URL (example, specify organism and genome)
main_page_url = "https://www.ncbi.nlm.nih.gov/nuccore?term=(Flaviviridae[Organism]%20AND%20complete[All%20Fields]%20AND%20genome[All%20Fields])%20AND%20(%22Homo%20sapiens%22[Organism]%20OR%20human[All%20Fields])&cmd=DetailsSearch"

# Function to extract genome links from a single page
def extract_genome_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all links on the page that lead to genome records
    links = [
        "https://www.ncbi.nlm.nih.gov" + a["href"]
        for a in soup.find_all("a", href=True)
        if "/nuccore/" in a["href"] and "advanced" not in a["href"]
    ]
    return links

# Function to check and extract next page URL (pagination)
def get_next_page_url(soup):
    next_button = soup.find("a", string="Next >")
    if next_button and "href" in next_button.attrs:
        return "https://www.ncbi.nlm.nih.gov" + next_button['href']
    else:
        return None

# Function to handle pagination and get all links
def get_all_genome_links(start_url):
    all_links = []
    current_url = start_url
    
    while current_url:
        # Extract links from the current page
        links = extract_genome_links(current_url)
        all_links.extend(links)
        
        # Fetch next page URL
        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, "html.parser")
        current_url = get_next_page_url(soup)
        
        # To prevent overwhelming the server
        time.sleep(1)
    
    return list(set(all_links))  # Remove duplicates, if any

# Fetch sequences for all extracted links and create a table
def fetch_sequences_table(links):
    data = []  # List to store rows of the table
    for link in links:
        try:
            # Extract accession ID from the link
            accession_id = link.split('/')[-1].split('?')[0]
            
            # Fetch the sequence
            handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="fasta", retmode="text")
            record = SeqIO.read(handle, "fasta")
            handle.close()
            
            # Add a row to the table
            data.append({
                "Sequence ID": record.id,
                "Description": record.description,
                "Sequence": str(record.seq)
            })
        except Exception as e:
            print(f"Error fetching from {link}: {e}")
    # Convert the list of rows to a DataFrame
    return pd.DataFrame(data)

# Get all genome links from the refined search
genome_links = get_all_genome_links(main_page_url)

# Fetch sequences and create a table
sequences_table = fetch_sequences_table(genome_links)

# Print the table size
print(f"Table Size: {sequences_table.shape}")  # Number of rows and columns

# Optionally save the table to a CSV file
sequences_table.to_csv("genome_sequences.csv", index=False)

Error fetching from https://www.ncbi.nlm.nih.gov/sites/myncbi/nuccore/filters: HTTP Error 400: Bad Request
Error fetching from https://www.ncbi.nlm.nih.gov/nuccore/details?querykey=1: HTTP Error 400: Bad Request
Table Size: (80, 3)


In [10]:
from Bio import Entrez, SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Function to fetch GenBank file using NCBI Entrez
def fetch_genbank_file(accession):
    Entrez.email = "smrithisurender@berkeley.edu"  # Provide your email for NCBI access
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
    genbank_record = SeqIO.read(handle, "genbank")
    handle.close()
    return genbank_record

# Function to convert GenBank annotation to GFF format
def genbank_to_gff(genbank_record):
    gff_data = []
    
    # Header line for GFF
    gff_data.append("##gff-version 3")
    
    # Extracting annotations from GenBank
    for feature in genbank_record.features:
        if feature.type in ["gene", "CDS", "mRNA"]:  # Focus on genes and CDS
            # GFF format fields: seqid, source, type, start, end, score, strand, phase, attributes
            seqid = genbank_record.id  # Use GenBank accession as seqid
            source = "GenBank"  # The source of the annotation
            feature_type = feature.type  # gene, CDS, mRNA
            start = feature.location.start + 1  # GFF is 1-based, GenBank is 0-based
            end = feature.location.end
            score = "."
            strand = "+" if feature.strand == 1 else "-"  # GFF uses + and - for strand
            phase = "."  # Phase is used for CDS, but we can leave it as "." for gene or mRNA
            attributes = f"ID={feature.id};Name={feature.qualifiers.get('gene', [''])[0]}"
            
            # GFF formatted line
            gff_line = [seqid, source, feature_type, str(start), str(end), score, strand, phase, attributes]
            gff_data.append("\t".join(gff_line))
    
    return gff_data

# Function to save GFF data to a file
def save_gff(gff_data, output_file="output.gff"):
    with open(output_file, "w") as f:
        for line in gff_data:
            f.write(line + "\n")

# Main execution
def main():
    accession = "KX528231"  # Example: SARS-CoV-2 genome (NCBI Accession ID)
    
    # Fetch GenBank file
    genbank_record = fetch_genbank_file(accession)
    
    # Convert GenBank annotations to GFF format
    gff_data = genbank_to_gff(genbank_record)
    
    # Save the GFF data to a file
    save_gff(gff_data)

    print(f"GFF file saved as 'output.gff'.")

# Run the script
if __name__ == "__main__":
    main()

GFF file saved as 'output.gff'.


/srv/conda/lib/python3.11/site-packages/Bio/SeqFeature.py:231: BiopythonDeprecationWarning: Please use .location.strand rather than .strand
  warnings.warn(
